In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))
print((torch.cuda._get_nvml_device_index(0)))

True
NVIDIA A100-SXM4-80GB
3


In [3]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFDirectoryLoader, WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [4]:
# 모델 경로 설정
root_path = Path(".").resolve().parent.parent
print(root_path)

/AIHUB/PCNRND/home/chatbot


In [5]:
# /AIHUB/PCNRND/home/chatbot/ai_models/base_models/embeddings/dragonkue/BGE-m3-ko
embedding_model_path = root_path / "ai_models" / "base_models" / "embeddings" /"dragonkue" / "BGE-m3-ko"

# 임베딩 모델 초기화
embeddings = HuggingFaceEmbeddings(
    model_name=str(embedding_model_path),
    model_kwargs={'device': 'cuda:0'},
    encode_kwargs={'normalize_embeddings': True}
)

/tmp/ipykernel_642703/41752180.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(


In [ ]:
# def load_pdf_directory(directory_path):
#     loader = PyPDFDirectoryLoader(directory_path)
#     pages = loader.load()
#     return pages

from langchain_community.document_loaders import WebBaseLoader

def load_web_data(url_list):
    loader = WebBaseLoader(url_list)
    pages = loader.load()

    # 줄바꿈 노이즈 정리
    for page in pages:
        # 하이픈으로 줄바꿈된 단어 복원
        page.page_content = page.page_content.replace("-\n", "")
        # 일반 줄바꿈은 공백으로 변환
        page.page_content = page.page_content.replace("\n", " ")

    return pages

web_urls = [
    "https://www.example.com",
    "https://www.example2.com"
]
web_data = load_web_data(web_urls)

PdfReadError("Invalid Elementary Object starting with b'\\xb5' @1042106: b'/BaseFont /\\xbf\\xa1\\xbd\\xba\\xc4\\xda\\xbe\\xee \\xb5\\xe5\\xb8\\xb2 9 Black\\n/Encoding /Identity-H\\n\\n/DescendantFonts [ 255 0'")
PdfReadError('Expecting a NameObject for key but found 9')
PdfReadError("Invalid Elementary Object starting with b'B' @1042113: b'nt /\\xbf\\xa1\\xbd\\xba\\xc4\\xda\\xbe\\xee \\xb5\\xe5\\xb8\\xb2 9 Black\\n/Encoding /Identity-H\\n\\n/DescendantFonts [ 255 0 R\\n ] \\n'")
PdfReadError("Invalid Elementary Object starting with b'\\xb5' @1100897: b'/BaseFont /\\xbf\\xa1\\xbd\\xba\\xc4\\xda\\xbe\\xee \\xb5\\xe5\\xb8\\xb2 9 Black \\n/CIDToGIDMap /Identity \\n/Subtype /Type1\\n/CIDSy'")
PdfReadError('Expecting a NameObject for key but found 9')
PdfReadError("Invalid Elementary Object starting with b'B' @1100904: b'nt /\\xbf\\xa1\\xbd\\xba\\xc4\\xda\\xbe\\xee \\xb5\\xe5\\xb8\\xb2 9 Black \\n/CIDToGIDMap /Identity \\n/Subtype /Type1\\n/CIDSystemInf'")
Ignoring wrong pointing object 14 

In [6]:

def split_documents(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=400,
        chunk_overlap=50,
        length_function=len,
        separators=[r"\n{2,}", r"\n", r"[.!?]", r"[,;:]", r" "],
        is_separator_regex=True
    )
    return text_splitter.split_documents(documents)

chunks = split_documents(pdf_data)

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=400,  # 한 청크에 너무 많은 문장이 담기지 않도록
#     chunk_overlap=50,  # 앞뒤 문맥 연결 위해 소폭 겹침
#     separators=[r"\n{2,}", r"\n", r"[.!?]", r"[,;:]", r" "],  # 문단, 줄, 문장, 쉼표, 공백 순으로 분할
#     is_separator_regex=True
# )

# chunks = text_splitter.split_documents(pdf_data)

# text_splitter = RecursiveCharacterTextSplitter(
#     chunk_size=500,  # PDF 문서는 더 작은 청크로 나누는 것이 좋음
#     chunk_overlap=50, # 청크 간 중복도 줄임
#     length_function=len,
#     separators=["\n\n", "\n", ".", "!", "?", ";", ":", " ", ""],  # PDF 문서의 구조를 고려한 구분자 추가
#     is_separator_regex=False
# )

# chunks = text_splitter.split_documents(pdf_data)

In [7]:
# FAISS 벡터 스토어 생성
db = FAISS.from_documents(
    chunks, 
    embeddings
)

In [8]:
# FAISS 벡터 스토어 저장
faiss_index_directory = "./faiss_index_directory"
# os.makedirs(faiss_index_directory, exist_ok=True)
db.save_local(faiss_index_directory)

In [9]:
# FAISS 벡터 스토어에서 유사도 검색
query = "피지컬 AI 란?"
docs = db.similarity_search(query, k=3)  # k는 반환할 문서 수

# 검색 결과 출력
for i, doc in enumerate(docs):
    print(f"\n## 검색결과 {i+1}")
    print(f"내용: {doc.page_content}")
    print(f"출처: {doc.metadata}")


## 검색결과 1
내용: .  - AI를 알고리즘만으로 만들지 않고, 몸과 환경을 통해 시각․청각․촉각․후각 등의 멀티 모달 센서로 현실 세계를 인식하고, '체험'할 수 있게 해서 더 현실적인 지능을 구현한다는 의미  - 즉, ‘몸을 가진 AI’로 물리적 환경 속에서 실시간으로 감각하고 반응하며 행동하는 AI 시스템을 의미하며, 시각․청각․촉각 등 다양한 센서로 외부 세계를 인식하고 그에 맞는 행동을 수행  ※ 신체화 AI 핵심 기술 요소: 강화학습(Reinforcement Learning), 로봇공학(Robotics), 컴퓨터 비전(Computer Vision), 자연어처리 (NLP), 시뮬레이션 플랫폼(AI 훈련용 가상 환경)
출처: {'producer': 'Hancom PDF 1.3.0.547', 'creator': 'Hwp 2018 10.0.0.14241', 'creationdate': '2025-04-18T15:01:47+09:00', 'author': '김종금', 'moddate': '2025-04-18T15:01:47+09:00', 'pdfversion': '1.4', 'source': '../data/pdf3/★_250402_(AI리포트)_(최종)_신체성_AI_현황과_전망.pdf', 'total_pages': 20, 'page': 7, 'page_label': '8'}

## 검색결과 2
내용: ‘AI 입은 로봇, 신체화 AI(Embodied AI)’ 현황과 전망 6 ○ 신체화 AI는 신체를 가진 AI를 의미하며, 물리적 신체와 지능형 에이전트를 갖추고 실세계에서 물체를 조작하거나 물리적 상호작용으로 인간과 소통할 수 있는 에이전트 기반의 AI 시스템  - 로봇과 같은 물리적 신체에 AI가 융합, 환경을 인지․학습해 동적으로 상호작용 할 수 있고, 계산․논리적 추론을 기반으로 지각․행동․환경 피드백을 통해 학습․적응해서 작업을 완료하는 시스템  ※ 휴머노이드 로봇 외에 자율주행차․웨어러블기기 등 상호작용 기능을 갖춘 HW도 신체화 AI의 본체 

In [10]:
faiss_index_directory = "./faiss_index_directory"
vectorstore = FAISS.load_local(faiss_index_directory, embeddings, allow_dangerous_deserialization=True)
retriever = vectorstore.as_retriever(    
    search_type="similarity_score_threshold", 
    search_kwargs={"score_threshold": 0.5, "k": 3}
)

In [11]:
vectorstore.similarity_search(k=3, query="ai 윤리 알려줘")

[Document(id='dce99223-e797-40db-a0f2-ea083d98402b', metadata={'producer': 'Hancom PDF 1.3.0.547', 'creator': 'Hwp 2020 11.0.0.8658', 'creationdate': '2024-12-23T08:29:00+09:00', 'moddate': '2024-12-23T08:29:00+09:00', 'pdfversion': '1.4', 'source': '../data/pdf3/[D.gov_이슈분석_2024-2호]_초연결사회의_양면성과_정책_과제.pdf', 'total_pages': 32, 'page': 19, 'page_label': '20'}, page_content='. (n.d.) AI 윤리기준 개요.'),
 Document(id='5e1d8821-0ee6-4e6c-bb09-480b35fae5e1', metadata={'producer': 'Hancom PDF 1.3.0.546', 'creator': 'Hwp 2020 11.0.0.8362', 'creationdate': '2024-07-25T10:14:05+09:00', 'author': '김종금', 'moddate': '2024-07-25T10:14:05+09:00', 'pdfversion': '1.4', 'source': '../data/pdf3/AI_REPORT_2024_1_2024년_AI_이슈를_용어와_함께_쉽게_이해하기_최종수정_게시용_20240725.pdf', 'total_pages': 175, 'page': 140, 'page_label': '141'}, page_content='.② 쉽게 설명하면?AI 윤리는 인공지능을 개발하고 사용하는 과정에서 우리가 지켜야 할 도덕적 기준이나 규범을 말한다. 마치 어린아이에게 옳고 그름을 가르치듯, 인공지능이 인간 사회에 긍정적 영향을 미치도록 바른 길로 이끄는 나침반 같은 역할을 한다. 예를 들어 ‘인공지능은 인간에게 해를 끼쳐서는 안 된다’, ‘인공지능의 판